<a href="https://colab.research.google.com/github/Czarczynski/API-ekspolracja-danych-projekt/blob/main/Seedlings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projekt dotyczył klasyfikowania sadzonek roślin na podstawie ich zdjęć. Dane były zapisane w dwóch folderach - train oraz test. Struktura folderu train składała się z dwunastu podfolderów nazwanych gatunkami roślin, które się w nich znajdowały. Folder test zawierał same zdjęcia bez etykiet, służył on do stworzenia submisji na portalu kaggle. W tym projekcie postanowiliśmy nauczyć naszą sieć z folderu train i nie wysyłać wyników na kaggle.

**Podsumowanie głównych bibliotek**

*   `os` służy jako interfejs do obsługi systemów operacyjnych, my go użyjemy do listowania folderów i plików;
*   `numpy` przyda się nam do konwersji danych;
*  `pandas` użyjemy do przeprowadzenia krótkiej analizy eksploracyjnej;
*   `cv2` służy do transformowania zdjęć;
*   `keras`, `sklearn` oraz `tensorflow` służą do obsługi modeli sieci neuronowych;
*   `matplotlib.pyplot` użyjemy do wyświetlenia zdjęć oraz wykresów zależności straty i dokładności od epochów;
*   `tqdm` służy do wyświetlenia progresu iteracji w pętli;
*   `google.colab.patches` naprawia błąd z wyświetlaniem zdjęć z biblioteki cv2.

Do prawidłowego działania tego programu należy pobrać dane wejściowe na dysk Google, podać do programu ścieżkę oraz podłączyć dysk do notatnika w Colabie.

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm

Poniższy chunk kodu służy do załadowania zdjęć do tablic. Zmienna path prowadzi nas do folderu, w których znajdują się podfoldery ze zdjęciami. Zmienna foldery tworzy listę wszystkich podfolderów, po których iterujemy w pętli, aby zapisać wszystkie zdjęcia do tablic. Zdjęcie jest trochę zmniejszone, aby zredukować czas ładowania oraz pamięć potrzebną do przechowania ich. Zapisujemy również etykietę danego zdjęcia biorąc nazwę folderu, w którym obecnie się znajdujemy. file_path służy jako kontrola, czy wszystko dobrze się zapisuje.
Zakomentowana komenda tworzy okrojony zbiór danych, w którym we wszystkich klasach jest tyle samo zdjęć. Będziemy również rozpatrywać model wytrenowany właśnie na danych uszczuplonych.

In [ ]:
path = "/content/drive/My Drive/Projekt_SN/train"
pwd = os.getcwd()
foldery = os.listdir(path) 
train_labels = []
train_images = []
file_path = []
i=1

for folder in tqdm(foldery):
    fol_path = path + '/'+ folder
    files = os.listdir(fol_path)
    for file in files:
    #for file in files[:220]:
        img_path = fol_path + '/' + file
        train_labels.append(folder)
        train_images.append(cv2.resize(cv2.imread(img_path), (64, 64)))
        file_path.append(img_path)

In [ ]:
len(train_images)

Dane zawierają 2640 zdjęć w przypadku zbioru okrojonego.

In [ ]:
print(np.unique(train_labels))
print(len(np.unique(train_labels)))

Mamy 12 gatunków sadzonek.

In [ ]:
df= pd.DataFrame(train_labels)
df.groupby([0]).size().sort_values(ascending=False)

Jak widać, dane są niezbalansowane. Dlatego też zdecydowaliśmy się na sprawdzenie, czy dane zrównoważone przynoszą lepsze efekty.

Poniżej zdefiniowana jest funkcja nakładająca maskę na zdjęcie. Zmienne `lower_green` i `upper_green` definiują granicę koloru zielonego w notacji HSV. 

Kolory w notacji HSV mają następujące parametry:
* H - hue, czyli odcień światła;
* S - saturation, czyli nasycenie koloru;
* V - value lub inaczej brightness, czyli moc światła białego.

Notacja HSV jest nam potrzebna ponieważ funkcja `cv2.inRange` wymaga na wejściu zdjęcia oraz granic właśnie w tym formacie. Jednocześnie, nie tracimy żadnych informacji o zdjęciu, więc możemy spokojnie wykonać taką transformację.

`image_hsv` konwertuje zdjęcie z RGB do HSV. `mask` definiuje maskę eliminując wszystkie kolory nie mieszczące się w granicy `lower` - `upper`. `img_mask` nakłada maskę na zdjęcie, a `img_gray` konwertuje zdjęcie z kolorowego do szarego, dzięki czemu wyeliminujemy dwa kanały kolorów i będziemy mieli przyjemniejsze wymiary danych.

Transformując zdjęcie do szarości po nałożeniu na nie maski nie tracimy żadnych informacji, gdyż na tym zdjęciu znajdują się tylko elementy o kolorze zielonym.


In [ ]:
def img_preprocessing(img):
  lower_green = np.array([25, 50, 50])
  upper_green = np.array([95, 255, 255])

  image_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  mask = cv2.inRange(image_hsv, lower_green, upper_green)
  img_mask = cv2.bitwise_or(img, img, mask = mask)
  img_gray = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)
  return img_gray

Poniższy chunk demonstruje działanie funkcji `img_preprocessing`.

In [ ]:
ind = 4500
plt.imshow(train_images[ind])
plt.show()
print(file_path[ind])
img = img_preprocessing(train_images[ind])
plt.imshow(img, cmap=plt.get_cmap("gray"))
plt.show()

Tutaj przeprowadzamy ostateczne przekształcenie naszych danych do formy, którą zadamy naszemu modelowi. Tworzymy tabelę trainX, która będzie zawierała informację o zdjęciach i trainY, która zawiera etykietę danego zdjęcia. Następnie przekształcamy tabelę z etykietami w wektor i aplikujemy One-Hot encoding. Transformacja ta polega na przedstawieniu klas jako wektory zero-jedynkowe, z jedynką w dokładnie jednym miejscu - odpowiadającym danej klasie. Następnie następuje przeskalowanie informacji o zdjęciach do przedziału $[0;1]$ oraz przetasowanie wartości. Przetasowanie jest potrzebne ponieważ przed nim wartości ustawione są 'po kolei', czyli najpierw są zdjęcia z pierwszej klasy, dalej z drugiej itd. Zbiór walidacyjny w modelu będziemy tworzyć poprzez parametr `validation_split`, więc przy uporządkowanym zbiorze danych walidacja odbywałaby się w klasach niewidzianych wcześniej przez model. Ostatnia linijka to dodanie wymiaru, konkretnie kanału koloru - w naszym przypadku mamy tylko jeden kanał, czyli szarość.

In [ ]:
from sklearn.utils import shuffle
trainX = []
trainY = []
for i in range(len(train_images)):
  trainX.append(img_preprocessing(train_images[i]))
  trainY.append(train_labels[i])
trainY = np.asarray(trainY)
trainY = trainY.reshape(-1,1)
ohe = OneHotEncoder(handle_unknown='ignore', sparse = False)
trainY = ohe.fit_transform(trainY)
trainX = np.asarray(trainX)
trainX = trainX/255
trainX, trainY = shuffle(trainX, trainY)
trainX = tf.expand_dims(trainX, axis=-1)

**Podsumowanie modelu** 

Do stworzenia algorytmu przewidującego gatunek rośliny na podstawie sadzonki zaprzęgliśmy model konwolucyjny. Dzięki niemu nie utracimy cennych informacji dotyczących zależności w sąsiedztwach poszczególnych pikseli. Poprzez metodę prób i błędów zdecydowaliśmy się na następującą strukturę, która daje najlepsze wyniki.

Parametr `padding` ustaliliśmy na wartość `same`, aby kernel nie uciął brzegów zdjęcia, a w przypadku niektórych sadzonek brzeg zdjęcia również ma znaczenie.

Ostatnia warstwa ma funkcję aktywacji `softmax`, aby wyliczyć prawdopodobieństwo przynależności do danej klasy.

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 48, kernel_size = (5,5), activation ='relu', padding = 'same'))
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', padding = 'same'))
model.add(Conv2D(filters = 16, kernel_size = (5,5), activation ='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling2D())
model.add(Dense(96, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(12, activation = "softmax"))
model.build(input_shape=(None,64,64,1))
model.summary()

Zaimplemeny został również early stopping, aby zapobiec przeuczeniu się sieci.

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

Wybraliśmy Adama jako nasz optymalizator oraz kategoryczną cross-entropię jako metrykę funkcji strat. Model ten kompiluje się całkiem długo. Na końcu zaprogramowaliśmy mechanizm zapisujący model do pliku, aby wygodnie się dzielić modelem.

In [ ]:
model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ["accuracy"])
history = model.fit(trainX, trainY, epochs = 150, batch_size = 50, validation_split = 0.3, callbacks=[es])
model.save("model_seedlings.h5")